# Exercise Guide #4: CGI and WSGI

## Project: Make your server run CGI scripts
Let's modify our static web server (which currently serves the files it's asked for) with a conditional that checks to see if the file is a python script - if it is, it runs it, and returns the output from that program.

You can either modify your existing server or copy and paste this code into a new Python file and modify it.

```
import socket

listener = socket.socket()
listener.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
listener.bind(('localhost', 8000))
listener.listen(5)

while True:
    s, addr = listener.accept()
    request = s.recv(1000)

    method, rest = request.split(b' ', 1)
    path, rest = rest.split(None, 1)

    s.send(b'HTTP/1.0 200 OK\n')
    s.send(b'\n')

    # run a script here

    s.close()
```

Useful bits of code:
    * `import subprocess` - this is the module that has all the tools we need for running programs from Python
    * `subprocess.check_output` - takes an array of arguments

## Project: Make your server run a WSGI application
A similar transformation: make your server call a function.

### Server combined with application

```python
import socket

def serve():
    listener = socket.socket()
    listener.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    listener.bind(('', 8080))
    listener.listen(5)
    while True:
        s, addr = listener.accept()
        request = s.recv(10000)
        print('request text:', request)
        method, rest = request.split(b' ', 1)
        path, rest = rest.split(None, 1)
        print('method:', method)
        print('path:', path)
        s.send(b'HTTP/1.1 200 OK\n\n')
        s.send('you asked to {} {}'.format(method, path).encode('utf-8'))
        s.close()

if __name__ == '__main__':
    serve()

```

### Server separated from applicaton, using the WSGI boundary

```python
import socket
# see wsgiref.simple_server for a real implementation of a WSGI server

def serve(app):
    listener = socket.socket()
    listener.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    listener.bind(('', 8080))
    listener.listen(5)
    while True:
        s, addr = listener.accept()
        print('server received connection from', addr)
        request = s.recv(10000)
        print('request we received:', request)
        method, rest = request.split(b' ', 1)
        path, rest = rest.split(None, 1)
        def start_response(status, headers):
            print('sending headers')
            headers = b'\r\n'.join([('HTTP/1.1 '+status).encode('ascii')] +
                                 [(k+': '+v).encode('ascii') for k, v in headers])
            print(headers)
            s.send(headers)
            s.send(b'\r\n\r\n')
        environ = {'REQUEST_METHOD': method.decode('ascii'),
                   "PATH_INFO": path.decode('ascii')}
        for data in app(environ, start_response):
            print('sending data')
            s.sendall(data.encode('utf-8'))
        s.close()

def demo_app(environ, start_response):
    start_response("200 OK", [('Content-Type','text/plain')])
    return ['You asked to '+environ['REQUEST_METHOD']+' '+environ['PATH_INFO'],
            ' and that is what we just did!']

if __name__ == '__main__':
    serve(demo_app)

```